# PyGaggle Hello World

In [1]:
!nvidia-smi

Thu Sep  2 08:21:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:01:00.0 Off |                    0 |
| N/A   19C    P0    49W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import json

queryToDocs = json.load(open('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/resources/all-top-100.json'))


In [8]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5
from pygaggle.rerank.transformer import MonoBERT

2021-08-05 17:27:14 [INFO] loader: Loading faiss with AVX2 support.
2021-08-05 17:27:14 [INFO] loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
2021-08-05 17:27:14 [INFO] loader: Loading faiss.
2021-08-05 17:27:15 [INFO] loader: Successfully loaded faiss.


In [9]:
monoBertReranker = MonoBERT()

In [10]:
monoT5Reranker = MonoT5()

In [20]:
docs_store = ir_datasets.load('msmarco-document-v2').docs_store()

In [5]:
import ir_datasets
from tqdm import tqdm

MS_MARCO_V2_QUERY_DATASETS = [
    ir_datasets.load('msmarco-document-v2/train'),
    ir_datasets.load('msmarco-document-v2/dev1'),
    ir_datasets.load('msmarco-document-v2/dev2'),
    ir_datasets.create_dataset(queries_tsv="/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/resources/2021_queries.tsv")
]

queries = {}
for query_dataset in MS_MARCO_V2_QUERY_DATASETS:
    for query in tqdm(query_dataset.queries_iter()):
        queries[query.query_id] = query.text


322196it [00:00, 323915.78it/s]
4552it [00:00, 134830.98it/s]
5000it [00:00, 118149.41it/s]
477it [00:00, 25087.56it/s]


In [28]:
def process_query(query_id, documents):
    documentFeatures = {i:{} for i in documents}
    
    query = queries[query_id]
    documents = [Text(docs_store.get(i), {'docid': i}, 0) for i in documents]
    if query is None or query.strip() == '':
        raise ValueError('Should not happen')
    if len(documents) != 100:
        raise ValueError('Should not happen')
    
    #for i in monoBertReranker.rerank(Query(query), documents):
    #    documentFeatures[i.metadata["docid"]]['feature_mono_bert'] = i.score

    for i in monoT5Reranker.rerank(Query(query), documents):
        documentFeatures[i.metadata["docid"]]['feature_mono_t5'] = i.score
        
    return {
        'query': query,
        'topic': query_id,
        'documentFeatures': documentFeatures
    }

In [ ]:
# Write the actual Data to 

In [ ]:
for query_id in tqdm(queryToDocs):
    json.dump(process_query(query_id, queryToDocs[query_id]), open('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/pygaggle_features/' + query_id + '.json', 'w'))

  0%|          | 13/332221 [02:13<680:11:38,  7.37s/it] /usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 2088/332221 [18:15:29<2423:31:03, 26.43s/it]  

In [25]:
process_query('1048595', queryToDocs['1048595'])

{'query': 'who plays snowden in the movie',
 'topic': '1048595',
 'documentFeatures': {'msmarco_doc_11_1566855042': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -9.774126052856445},
  'msmarco_doc_34_894540006': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -12.008002281188965},
  'msmarco_doc_25_13209253': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -7.794625282287598},
  'msmarco_doc_06_1679863215': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -8.38987922668457},
  'msmarco_doc_17_1201364868': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -13.23913860321045},
  'msmarco_doc_36_933724560': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -11.638420104980469},
  'msmarco_doc_18_2995196448': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -0.08372984826564789},
  'msmarco_doc_56_956040808': {'feature_mono_bert': -11.574138641357422,
   'feature_mono_t5': -1

In [27]:
for i in monoT5Reranker.rerank(Query('who plays snowden in the movie'), [Text(docs_store.get('msmarco_doc_11_1566855042'), {'docid': 'msmarco_doc_11_1566855042'}, 0), Text(docs_store.get('msmarco_doc_18_2995196448'), {'docid': 'msmarco_doc_18_2995196448'}, 0)]):
    print(i.metadata["docid"] + ' -> ' + str(i.score))

msmarco_doc_11_1566855042 -> -9.775344848632812
msmarco_doc_18_2995196448 -> -0.08368939161300659
